<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [3]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [4]:
# Posibles bandas
os.listdir("./songs_dataset/")

['bruce-springsteen.txt',
 'adele.txt',
 'dr-seuss.txt',
 'missy-elliott.txt',
 'kanye-west.txt',
 'radiohead.txt',
 'nickelback.txt',
 'Kanye_West.txt',
 'dickinson.txt',
 'blink-182.txt',
 'notorious_big.txt',
 'disney.txt',
 'paul-simon.txt',
 'beatles.txt',
 'cake.txt',
 'dj-khaled.txt',
 'ludacris.txt',
 'janisjoplin.txt',
 'michael-jackson.txt',
 'lady-gaga.txt',
 'kanye.txt',
 'prince.txt',
 'bob-marley.txt',
 'bjork.txt',
 'johnny-cash.txt',
 'britney-spears.txt',
 'al-green.txt',
 'patti-smith.txt',
 'nicki-minaj.txt',
 'drake.txt',
 'bieber.txt',
 'nursery_rhymes.txt',
 'r-kelly.txt',
 'joni-mitchell.txt',
 'Lil_Wayne.txt',
 'dolly-parton.txt',
 'jimi-hendrix.txt',
 'lorde.txt',
 'amy-winehouse.txt',
 'bob-dylan.txt',
 'nirvana.txt',
 'bruno-mars.txt',
 'rihanna.txt',
 'lil-wayne.txt',
 'eminem.txt',
 'lin-manuel-miranda.txt',
 'alicia-keys.txt',
 'leonard-cohen.txt',
 'notorious-big.txt']

In [3]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
#df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)

df = pd.read_csv('/content/sample_data/corpus.txt', sep='/n', header=None)
df.head(200)

<ipython-input-3-975fd73aad19>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/sample_data/corpus.txt', sep='/n', header=None)


,0
0,"Cuando estés muy lejos de la pared, pero te es..."
1,"si te estas acercando a la pared, pero aún est..."
2,En caso de estar muy lejos de la pared y estás...
3,En el caso de estar demasiado lejos de la pare...
4,"Cuando estés lejos de la pared, pero te estás ..."
5,si te estás acercando a la pared pero aún está...
6,en caso de que te estés acercando al pared per...
7,en caso de estar lejos de la pared pero te est...
8,"Cuando estés a la distancia requerida, pero te..."
9,"si estas en la distancia que es, pero te vas a..."


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 60


### 1 - Preprocesamiento

In [5]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [6]:
# Demos un vistazo
sentence_tokens[:2]

[['cuando',
  'estés',
  'muy',
  'lejos',
  'de',
  'la',
  'pared',
  'pero',
  'te',
  'estás',
  'acercando',
  'a',
  'ella',
  'gira',
  'tu',
  'rueda',
  'izquierda',
  'rápidamente',
  'hacia',
  'delante',
  'y',
  'gira',
  'tu',
  'rueda',
  'derecha',
  'muy',
  'lento',
  'hacia',
  'atrás'],
 ['si',
  'te',
  'estas',
  'acercando',
  'a',
  'la',
  'pared',
  'pero',
  'aún',
  'estás',
  'demasiado',
  'lejos',
  'de',
  'ella',
  'gira',
  'tu',
  'rueda',
  'izquierda',
  'rápidamente',
  'hacia',
  'adelante',
  'y',
  'la',
  'rueda',
  'derecha',
  'lentamente',
  'hacia',
  'atrás']]

### 2 - Crear los vectores (word2vec)

In [133]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [183]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=1,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=15,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [184]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [185]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 60


In [186]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 94


### 3 - Entrenar el modelo generador

In [187]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=30,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 16069.677734375
Loss after epoch 1: 6675.951171875
Loss after epoch 2: 5604.9609375
Loss after epoch 3: 5483.484375
Loss after epoch 4: 5885.48828125
Loss after epoch 5: 5867.609375
Loss after epoch 6: 5774.37109375
Loss after epoch 7: 5829.17578125
Loss after epoch 8: 5717.94140625
Loss after epoch 9: 5468.35546875
Loss after epoch 10: 5406.171875
Loss after epoch 11: 5733.65625
Loss after epoch 12: 5575.03125
Loss after epoch 13: 5575.2109375
Loss after epoch 14: 5587.3515625
Loss after epoch 15: 6042.578125
Loss after epoch 16: 4997.2109375
Loss after epoch 17: 5125.953125
Loss after epoch 18: 5542.8515625
Loss after epoch 19: 5275.140625
Loss after epoch 20: 6007.5625
Loss after epoch 21: 5118.703125
Loss after epoch 22: 5384.03125
Loss after epoch 23: 5238.75
Loss after epoch 24: 5028.71875
Loss after epoch 25: 5482.53125
Loss after epoch 26: 5409.5625
Loss after epoch 27: 5350.140625
Loss after epoch 28: 5129.484375
Loss after epoch 29: 5252.046875


(16233, 53340)

### 4 - Ensayar

In [188]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["acercando"], topn=10)

[('lejos', 0.9954859614372253),
 ('si', 0.9925446510314941),
 ('a', 0.9890255928039551),
 ('estar', 0.9883588552474976),
 ('estás', 0.9881230592727661),
 ('cerca', 0.987986147403717),
 ('cuando', 0.9879797101020813),
 ('aún', 0.9873345494270325),
 ('alejando', 0.9869476556777954),
 ('pero', 0.9864088892936707)]

In [189]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["pared"], topn=10)

[('hacia', -0.5032098889350891),
 ('derecha', -0.5390797853469849),
 ('rueda', -0.5432543158531189),
 ('rápido', -0.5518162250518799),
 ('delante', -0.5688991546630859),
 ('tu', -0.5773341655731201),
 ('atrás', -0.5809046626091003),
 ('izquierda', -0.6039652824401855),
 ('adelante', -0.6101663112640381),
 ('lento', -0.6242133378982544)]

In [190]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["derecha"], topn=10)

[('tu', 0.9925791621208191),
 ('izquierda', 0.991791844367981),
 ('delante', 0.9910332560539246),
 ('rápido', 0.9881554245948792),
 ('hacia', 0.985762894153595),
 ('rueda', 0.9844350814819336),
 ('lento', 0.9825069308280945),
 ('atrás', 0.9820074439048767),
 ('adelante', 0.9810807704925537),
 ('despacito', 0.9790036678314209)]

In [191]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["distancia"], topn=5)

[('que', 0.9872562289237976),
 ('cuando', 0.9871909022331238),
 ('estés', 0.98658287525177),
 ('estar', 0.9853087067604065),
 ('pero', 0.9848620295524597)]

In [26]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: ignored

### 5 - Visualizar agrupación de vectores

In [153]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [192]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

# Explicación del dataset utilizado

Se utiliza un dtaset generado manualmente que contiene un conjunto de instrucciones se como realizar una tarea de seguimiento de pared, en total son 15 parrafos, cada 1 de ellos cuenta con una frase contextual con una estructura canónica en la que cambian algunos terminos clave, las tres frases siguientes solo cambia su sintaxis y no necesarimente sigue esa estructura canónica pero contiene aquellas palabras clave

## Conclusiones

- Para el conjunto de datos en cuestión SkipGram da una mejor comparativa del contexto de las palabras en un plano cartesiano 2D. puede contener el contexto de "alejarse", "alejandose" o "lejos"  en un eje del plano mientras el contexto de estar "cerca" o "acercandose" llos generaliza al otro lado del plano.


- CBOW es más disperso, genera similitudes no muy acertadas o no relaciona de alguna forma muy bien el contexto.

- muchas epochs hacen que ambos modelos entreguen resultados considerados muy errados
